# 1. Data preprocessing

The goal of this worksheet is to fully process the dataset so it can be used for modeling.

This worksheet contains the following steps:
1. Obtaining the data
2. Inspecting the data
3. Cleaning and filtering the data
4. Feature engineering
5. Creating the target variable
6. Data selection for modeling


In [98]:
import pandas as pd
from requests import get
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Obtaining the data

The dataset is quite large so it should be only downloaded once. However if you work on it continuously then consider re-downloading it once per month, as Bondora adds more loans and some of the old loans are updated. After downloading we extract the zip file and load the extracted .csv file into a DataFrame and also sort it in a chronological order.

In [127]:
def download():

    import zipfile
    url = "https://bondora.com/marketing/media/LoanData.zip"

    with open("data/Data.zip", "wb") as file:
        response = get(url)
        file.write(response.content)

    with zipfile.ZipFile("data/Data.zip","r") as zip_ref:
        zip_ref.extractall("data/")

#download()

In [ ]:
data = pd.read_csv("LoanData.csv")
data = data.sort_values(by=['ListedOnUTC'])

## Inspecting the data

We can see that the dataset contains 192771 loans and has 112 different columns. Loans range from 2009 to the current date.

In [100]:
print(f"DataFrame shape: {data.shape}")

DataFrame shape: (192771, 112)


In [101]:
data.head(10)

,ReportAsOfEOD,LoanId,LoanNumber,ListedOnUTC,BiddingStartedOn,BidsPortfolioManager,BidsApi,BidsManual,UserName,NewCreditCustomer,LoanApplicationStartedDate,LoanDate,ContractEndDate,FirstPaymentDate,MaturityDate_Original,MaturityDate_Last,ApplicationSignedHour,ApplicationSignedWeekday,VerificationType,LanguageCode,Age,DateOfBirth,Gender,Country,AppliedAmount,Amount,Interest,LoanDuration,MonthlyPayment,County,City,UseOfLoan,Education,MaritalStatus,NrOfDependants,EmploymentStatus,EmploymentDurationCurrentEmployer,EmploymentPosition,WorkExperience,OccupationArea,HomeOwnershipType,IncomeFromPrincipalEmployer,IncomeFromPension,IncomeFromFamilyAllowance,IncomeFromSocialWelfare,IncomeFromLeavePay,IncomeFromChildSupport,IncomeOther,IncomeTotal,ExistingLiabilities,LiabilitiesTotal,RefinanceLiabilities,DebtToIncome,FreeCash,MonthlyPaymentDay,ActiveScheduleFirstPaymentReached,PlannedPrincipalTillDate,PlannedInterestTillDate,LastPaymentOn,CurrentDebtDaysPrimary,DebtOccuredOn,CurrentDebtDaysSecondary,DebtOccuredOnForSecondary,ExpectedLoss,LossGivenDefault,ExpectedReturn,ProbabilityOfDefault,DefaultDate,PrincipalOverdueBySchedule,PlannedPrincipalPostDefault,PlannedInterestPostDefault,EAD1,EAD2,PrincipalRecovery,InterestRecovery,RecoveryStage,StageActiveSince,ModelVersion,Rating,EL_V0,Rating_V0,EL_V1,Rating_V1,Rating_V2,Status,Restructured,ActiveLateCategory,WorseLateCategory,CreditScoreEsMicroL,CreditScoreEsEquifaxRisk,CreditScoreFiAsiakasTietoRiskGrade,CreditScoreEeMini,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalWriteOffs,InterestAndPenaltyWriteOffs,PrincipalBalance,InterestAndPenaltyBalance,NoOfPreviousLoansBeforeLoan,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PreviousEarlyRepaymentsCountBeforeLoan,GracePeriodStart,GracePeriodEnd,NextPaymentDate,NextPaymentNr,NrOfScheduledPayments,ReScheduledOn,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,ActiveLateLastPaymentCategory
957,2021-10-20,FA160D69-2682-4A60-8D8E-9BB700EA30CE,37,2009-02-21 14:12:39,2009-02-21 14:12:39,0,0.00,63.91,BO57KKKA,True,2009-02-21 14:12:39,2009-03-07,2009-09-10,2009-04-10,2009-09-10,2009-09-10,14,7,2.00,1,21,NaN,0.00,EE,63.91,63.91,20.00,6,NaN,NaN,NaN,3,5.00,4.00,0.00,5.00,UpTo2Years,NaN,5To10Years,NaN,NaN,15500.00,0.00,0.00,0.00,0.00,0.00,0.00,15500.00,0,0.00,0,0.00,0.00,10,True,1000.00,34.14,2009-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,NaN,NaN,NaN,NaN,NaN,NaN,63.91,2.18,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN
958,2021-10-20,8E929B92-7C99-421D-8499-9BB70125F390,42,2009-02-21 17:50:14,2009-02-21 17:50:14,0,0.00,83.09,Maret,True,2009-02-21 17:50:14,2009-03-03,2011-03-14,2009-04-13,2011-03-14,2011-03-14,17,7,2.00,1,19,NaN,1.00,EE,958.67,83.08,10.00,24,NaN,NaN,NaN,6,2.00,3.00,2.00,3.00,UpTo1Year,NaN,LessThan2Years,NaN,NaN,4000.00,0.00,0.00,0.00,0.00,0.00,0.00,4000.00,0,0.00,0,0.00,0.00,12,True,1300.00,142.73,2011-03-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,NaN,180+,NaN,NaN,NaN,NaN,83.08,9.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN
959,2021-10-20,33B3F669-D0E3-4474-8045-9BB70128D064,43,2009-02-21 18:00:40,2009-02-21 18:00:40,0,0.00,322.75,OUPEN,True,2009-02-21 18:00:40,2009-02-28,2010-03-10,2009-04-10,2010-03-10,2010-03-10,18,7,2.00,1,38,NaN,0.00,EE,639.12,322.75,25.00,12,NaN,NaN,NaN,6,4.00,4.00,4.00,3.00,UpTo4Years,NaN,15To25Years,NaN,NaN,9000.00,0.00,0.00,0.00,0.00,0.00,0.00,9000.00,0,0.00,0,0.00,0.00,10,True,5050.00,697.28,2010-03-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,NaN,151-180,NaN,NaN,NaN,NaN,322.75,44.56,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN
960,2021-10-20,7074D9E8-E8F5-403B-8614-9BB701338AD4,44,2009-02-21 18:39:43,2009-02-21

In [102]:
data.tail(10)

,ReportAsOfEOD,LoanId,LoanNumber,ListedOnUTC,BiddingStartedOn,BidsPortfolioManager,BidsApi,BidsManual,UserName,NewCreditCustomer,LoanApplicationStartedDate,LoanDate,ContractEndDate,FirstPaymentDate,MaturityDate_Original,MaturityDate_Last,ApplicationSignedHour,ApplicationSignedWeekday,VerificationType,LanguageCode,Age,DateOfBirth,Gender,Country,AppliedAmount,Amount,Interest,LoanDuration,MonthlyPayment,County,City,UseOfLoan,Education,MaritalStatus,NrOfDependants,EmploymentStatus,EmploymentDurationCurrentEmployer,EmploymentPosition,WorkExperience,OccupationArea,HomeOwnershipType,IncomeFromPrincipalEmployer,IncomeFromPension,IncomeFromFamilyAllowance,IncomeFromSocialWelfare,IncomeFromLeavePay,IncomeFromChildSupport,IncomeOther,IncomeTotal,ExistingLiabilities,LiabilitiesTotal,RefinanceLiabilities,DebtToIncome,FreeCash,MonthlyPaymentDay,ActiveScheduleFirstPaymentReached,PlannedPrincipalTillDate,PlannedInterestTillDate,LastPaymentOn,CurrentDebtDaysPrimary,DebtOccuredOn,CurrentDebtDaysSecondary,DebtOccuredOnForSecondary,ExpectedLoss,LossGivenDefault,ExpectedReturn,ProbabilityOfDefault,DefaultDate,PrincipalOverdueBySchedule,PlannedPrincipalPostDefault,PlannedInterestPostDefault,EAD1,EAD2,PrincipalRecovery,InterestRecovery,RecoveryStage,StageActiveSince,ModelVersion,Rating,EL_V0,Rating_V0,EL_V1,Rating_V1,Rating_V2,Status,Restructured,ActiveLateCategory,WorseLateCategory,CreditScoreEsMicroL,CreditScoreEsEquifaxRisk,CreditScoreFiAsiakasTietoRiskGrade,CreditScoreEeMini,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalWriteOffs,InterestAndPenaltyWriteOffs,PrincipalBalance,InterestAndPenaltyBalance,NoOfPreviousLoansBeforeLoan,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PreviousEarlyRepaymentsCountBeforeLoan,GracePeriodStart,GracePeriodEnd,NextPaymentDate,NextPaymentNr,NrOfScheduledPayments,ReScheduledOn,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,ActiveLateLastPaymentCategory
191981,2021-10-20,814566B1-0023-44B3-8F72-ADC50185CD50,2405917,2021-10-19 18:59:27,2021-10-19 21:59:27,274,0.00,4.00,BO7451973A,False,2021-10-18 23:39:14,2021-10-19,NaN,2021-11-15,2026-10-15,2026-10-15,23,2,4.00,4,38,NaN,1.00,FI,3943.00,3943.00,18.72,60,98.58,NaN,NaN,-1,3.00,-1.00,NaN,-1.00,UpTo5Years,NaN,NaN,-1.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1800.00,2,138.75,0,0.00,0.00,15,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09,0.79,0.07,0.12,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,D,NaN,NaN,NaN,NaN,NaN,Current,False,NaN,NaN,M,NaN,5,NaN,0.00,0.00,NaN,NaN,3943.00,NaN,1.00,4150.00,507.22,NaN,0.00,NaN,NaN,2021-11-15,1.00,60.00,NaN,NaN,NaN,NaN
192241,2021-10-20,48CC95D8-C950-4331-A336-ADC6014DCE7A,2407633,2021-10-19 19:04:03,2021-10-19 22:04:03,34,3.00,114.00,BO96429K3A,False,2021-10-19 20:15:21,2021-10-19,NaN,2021-11-15,2026-10-13,2026-10-13,20,3,1.00,1,24,NaN,0.00,EE,2126.00,2126.00,43.21,60,84.60,NaN,NaN,-1,4.00,-1.00,NaN,-1.00,UpTo1Year,NaN,NaN,-1.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1100.00,5,495.57,0,0.00,0.00,13,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.19,0.60,0.09,0.31,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,F,NaN,NaN,NaN,NaN,NaN,Current,False,NaN,NaN,M,NaN,NaN,700.00,0.00,0.00,NaN,NaN,2126.00,NaN,2.00,7442.00,1072.06,NaN,0.00,NaN,NaN,2021-11-15,1.00,60.00,NaN,NaN,NaN,NaN
192258,2021-10-20,3EE5ED96-E7F6-4D21-AF20-ADC6016BA019,2407758,2021-10-19 19:06:34,2021-10-19 22:06:34,2377,176.00,1677.00,BO92K97K2,False,2021-10-19 22:03:55,2021-10-19,NaN,2021-11-10,2026-10-12,2026-10-12,22,3,4.00,1,32,NaN,0.00,EE,5316.00,5316.00,28.99,60,176.13,NaN,NaN,-1,3.00,-1.00,NaN,-1.00,UpTo5Years,NaN,NaN,-1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1600.00,5,1006.33,0,0.00,0.00,10,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09,0.59,0.12,0.16,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,D,NaN,NaN,NaN,NaN,NaN,Current,False,NaN,NaN,M,NaN,NaN,900.00,0.00,0.00,NaN,NaN,5316.00,NaN,3.00,15203.00,4550.89,NaN,0.00,NaN,NaN,2021-11-10,1.00,60.00,NaN,NaN,NaN,NaN
192242,2021-10-20,92F130ED-F8A1-40C4-B933-ADC6014E71B

The data contains various numerical and categorical columns. To fully understand the data and proceed with cleaning the dataset, one must work through most of the columns. Visual inspection is important when working with this dataset as some columns have become obsolete halfway through the dataset. It is also helpful to statistically describe different columns as it helps to get a better understanding of the data. Below are some examples on how some of the numerical and categorical columns were analyzed.

In [103]:
# Describing a numerical column
data.Amount.describe()

count   192771.00
mean      2564.41
std       2198.90
min          6.39
25%        740.00
50%       2125.00
75%       4000.00
max      10632.00
Name: Amount, dtype: float64

In [104]:
# Numerical column value counts
data.Amount.value_counts(dropna=False)

530.00     20337
531.00     13669
4150.00    10337
2125.00     9046
2126.00     6827
           ...  
923.00         1
63.81          1
2708.00        1
671.09         1
1281.89        1
Name: Amount, Length: 6543, dtype: int64

In [105]:
# Describing a categorical column
data.Rating.describe()

count     190038
unique         8
top            D
freq       43741
Name: Rating, dtype: object

In [106]:
# Categorical column value counts
data.Rating.value_counts(dropna=False)

D      43741
E      34137
C      31077
F      26000
B      24232
HR     14832
A       8565
AA      7454
NaN     2733
Name: Rating, dtype: int64

## Cleaning and filtering the data

The first step is to drop all the unnecessary columns. Many of these columns are obsolete (see Bondora API docs) or serve no purpose for us. However some of the columns are not marked as obsolete in the API docs and half-way through the dataset they are filled only with null values. The reason behind this is most likely EU's data protection law (GDPR). These columns were detected by visually going through the dataset and since the last half of the data is missing in these columns, there is really no point in keeping them.

In [107]:
# Drop unnecessary columns.
df = data.drop(['BiddingStartedOn', 'LoanApplicationStartedDate', 'ApplicationSignedHour',
                'ApplicationSignedWeekday', 'DateOfBirth', 'County', 'City', 'UseOfLoan',
                'MaritalStatus', 'NrOfDependants', 'EmploymentStatus', 'EmploymentPosition',
                'WorkExperience', 'OccupationArea', 'IncomeFromPrincipalEmployer', 'IncomeFromPension',
                'IncomeFromFamilyAllowance', 'IncomeFromSocialWelfare', 'IncomeFromLeavePay',
                'IncomeFromChildSupport', 'IncomeOther', 'RefinanceLiabilities', 'DebtToIncome', 'FreeCash', 'MonthlyPaymentDay',
                'EL_V0', 'Rating_V0', 'EL_V1', 'Rating_V1', 'Rating_V2', 'PrincipalWriteOffs', 'InterestAndPenaltyWriteOffs', 'PlannedPrincipalTillDate', 'CreditScoreEsEquifaxRisk', 'CreditScoreEsMicroL', 'BidsPortfolioManager', 'BidsApi', 'BidsManual', 'PrincipalDebtServicingCost', 'InterestAndPenaltyDebtServicingCost'], axis=1)

In order to further reduce the size of the dataset we apply some filters. Some columns are parsed into date format, so it makes it easier to use them in the filters.

We also do not want currently active loans, as our goal is to classify if the loans are worth to invest in or not, and we do not know the outcome of those loans yet. However if the loan is active but has been restructured, then we keep it and use it in the target variable as a "bad" loan since we know for sure that the loan has had issues. The reason for not including loans that are active and have not had any issues is that we have no way of knowing if issues would or would not arise in the future.

We also apply some filters to numerical and categorical columns in order to deal with anomalies. And we also deal with null values.

In [108]:
# Parse date columns into date format.
date_columns = ['ListedOnUTC', 'LoanDate', 'ContractEndDate', 'MaturityDate_Original', 'MaturityDate_Last', 'ReportAsOfEOD']
df[date_columns] = df[date_columns].apply(pd.to_datetime, format='%Y-%m-%d', errors='coerce')

# Do not include loans that are currently active, and have not been restructured (the original maturity date of the loan has not increased by at least 60 days).
df = df.loc[~((df.Status == 'Current') & (df.Restructured == 0))]

# Some loans have been repaid just days after signing a contract so they should be filtered out.
df = df.loc[~(((df.MaturityDate_Last - df.LoanDate) < pd.to_timedelta("180days")) & (df.Status == 'Repaid'))]

# Remove the most recent loans because most of them seem to be with the status "Late".
df = df.loc[(df.LoanDate < (df.ReportAsOfEOD - pd.to_timedelta("60days")))]

In [109]:
# Filters regarding numerical values

# Age between 18 and 70.
df = df.loc[(df.Age >= 18) & (df.Age <= 70)]

# Monthly payment is over 0.
df = df.loc[(df.MonthlyPayment > 0)]

In [110]:
# Filters regarding categorical values

# Select allowed loan durations. This dataset contains various contract lengths, but we are interested in the most common ones. Values represent loan duration in months.
df.LoanDuration = df.LoanDuration.astype('category')
df = df.loc[(df.LoanDuration.isin([60, 48, 36, 30, 24, 18, 12, 9, 6]))]

# We only want loans from EE.
df = df.loc[(df.Country.isin(['EE']))]

# The main verification types.
df.VerificationType = df.VerificationType.astype('category')
df = df.loc[(df.VerificationType.isin([1, 2, 3, 4]))]

# Gender is either male of female.
df.Gender = df.Gender.astype('category')
df = df.loc[(df.Gender.isin([0, 1]))]

# Education
df.Education = df.Education.astype('category')
df = df.loc[(df.Education.isin([1, 2, 3, 4, 5]))]

# Employment duration
df = df.loc[(df.EmploymentDurationCurrentEmployer.isin(['MoreThan5Years', 'UpTo5Years', 'UpTo4Years', 'UpTo3Years', 'UpTo2Years', 'UpTo1Year', 'TrialPeriod', 'Retiree', 'Other']))]

# Home ownership
df.HomeOwnershipType = df.HomeOwnershipType.astype('category')
df = df.loc[(df.HomeOwnershipType.isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))]

# Rating
df = df.loc[(df.Rating.isin(['AA', 'A', 'B', 'C', 'D', 'E', 'F', 'HR']))]

# Credit score
df.CreditScoreEeMini = df.CreditScoreEeMini.astype('category')
df = df.loc[(df.CreditScoreEeMini.isin([1000, 900, 800, 700, 600, 500]))]

In [111]:
# Dealing with null values

df.PreviousRepaymentsBeforeLoan = df.PreviousRepaymentsBeforeLoan.fillna(0)

df.PreviousEarlyRepaymentsBefoleLoan = df.PreviousEarlyRepaymentsBefoleLoan.fillna(0)

In [112]:
print(f"DataFrame shape after cleaning: {df.shape}")

DataFrame shape after cleaning: (80918, 72)


## Feature engineering

TODO

## Constructing the target variable

We construct a target variable called "PreferLoan" which is a boolean value. As stated before, loans with previous issues are not to be preferred and are assigned the value "0". Loans without issues are assigned the value "1". However there is a small exception. If the the worst late category is 16-30 days or less and the loan has not had any other issues then the loan is also preferred.

In [ ]:
# Constructing the target value.

# Loan status must be 'Repaid'.
# WorseLateCategory must not be higher than 16-30 (can be null).
# Loan must be repaid before or on the original maturity date.
# Loan must not be restructured.
# Loan must not be defaulted.

# Set the default value for all loans to be 0.
df["PreferLoan"] = 0

# Select preferred loans and set their value to 1.
df.loc[(
               (df.Status == 'Repaid') &
               (df.WorseLateCategory.isin([np.nan, '1-7', '8-15', '16-30'])) &
               (df.MaturityDate_Last <= df.MaturityDate_Original) &
               (df.Restructured != 1) &
               (df.DefaultDate.isnull())
       ), 'PreferLoan'] = 1

## Data selection for modeling

We used many columns for filtering that have no use for our next step - creating a classification model. Below we select the columns that can be used for the modeling step. We also check for null values as some models can not deal with them. And we also have to one-hot encode the categorical values.

In [114]:
# We do not include the "Country" column, because we predict only loans from EE.
modeling_cols = ['NewCreditCustomer', 'VerificationType', 'LanguageCode', 'Age', 'Gender',
        'Amount', 'Interest', 'LoanDuration', 'MonthlyPayment',
        'Education', 'EmploymentDurationCurrentEmployer', 'HomeOwnershipType', 'IncomeTotal',
        'ExistingLiabilities', 'LiabilitiesTotal', 'Rating',
        'CreditScoreEeMini', 'NoOfPreviousLoansBeforeLoan', 'AmountOfPreviousLoansBeforeLoan',
        'PreviousRepaymentsBeforeLoan', 'PreviousEarlyRepaymentsBefoleLoan', 'PreferLoan']

df = df[modeling_cols]

In [115]:
# Check for null values
df.isna().sum()

NewCreditCustomer                    0
VerificationType                     0
LanguageCode                         0
Age                                  0
Gender                               0
Amount                               0
Interest                             0
LoanDuration                         0
MonthlyPayment                       0
Education                            0
EmploymentDurationCurrentEmployer    0
HomeOwnershipType                    0
IncomeTotal                          0
ExistingLiabilities                  0
LiabilitiesTotal                     0
Rating                               0
CreditScoreEeMini                    0
NoOfPreviousLoansBeforeLoan          0
AmountOfPreviousLoansBeforeLoan      0
PreviousRepaymentsBeforeLoan         0
PreviousEarlyRepaymentsBefoleLoan    0
PreferLoan                           0
dtype: int64

In [116]:
# One-hot encoding.
df = pd.get_dummies(df)
df

,NewCreditCustomer,LanguageCode,Age,Amount,Interest,MonthlyPayment,IncomeTotal,ExistingLiabilities,LiabilitiesTotal,NoOfPreviousLoansBeforeLoan,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PreferLoan,VerificationType_1.0,VerificationType_3.0,VerificationType_4.0,Gender_0.0,Gender_1.0,Gender_2.0,LoanDuration_3,LoanDuration_6,LoanDuration_9,LoanDuration_12,LoanDuration_18,LoanDuration_24,LoanDuration_30,LoanDuration_36,LoanDuration_42,LoanDuration_48,LoanDuration_54,LoanDuration_60,Education_-1.0,Education_1.0,Education_2.0,Education_3.0,Education_4.0,Education_5.0,EmploymentDurationCurrentEmployer_MoreThan5Years,EmploymentDurationCurrentEmployer_Other,EmploymentDurationCurrentEmployer_Retiree,EmploymentDurationCurrentEmployer_TrialPeriod,EmploymentDurationCurrentEmployer_UpTo1Year,EmploymentDurationCurrentEmployer_UpTo2Years,EmploymentDurationCurrentEmployer_UpTo3Years,EmploymentDurationCurrentEmployer_UpTo4Years,EmploymentDurationCurrentEmployer_UpTo5Years,HomeOwnershipType_1.0,HomeOwnershipType_2.0,HomeOwnershipType_3.0,HomeOwnershipType_4.0,HomeOwnershipType_5.0,HomeOwnershipType_6.0,HomeOwnershipType_7.0,HomeOwnershipType_8.0,HomeOwnershipType_9.0,HomeOwnershipType_10.0,Rating_A,Rating_AA,Rating_B,Rating_C,Rating_D,Rating_E,Rating_F,Rating_HR,CreditScoreEeMini_500.0,CreditScoreEeMini_600.0,CreditScoreEeMini_700.0,CreditScoreEeMini_800.0,CreditScoreEeMini_900.0,CreditScoreEeMini_1000.0
11326,True,1,52,700.00,26.00,88.48,590.00,4,626.00,0.00,0.00,0.00,0.00,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
11294,False,1,44,500.00,30.00,17.64,1195.00,9,721.50,1.00,1000.00,123.30,0.00,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
11346,True,3,43,4150.00,31.00,190.57,1386.00,3,831.00,0.00,0.00,0.00,0.00,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
11342,False,1,35,3500.00,32.00,133.62,1298.00,5,1133.82,1.00,3000.00,556.53,0.00,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
11351,True,1,39,1000.00,34.00,74.60,1033.00,2,670.00,0.00,0.00,0.00,0.00,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183394,True,3,25,2658.00,11.39,66.45,1600.00,2,363.57,1.00,531.00,180.79,0.00,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
183567,True,1,20,106.00,36.57,3.88,500.00,1,19.33,1.00,531.00,34.44,0.00,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
183568,False,1,33,956.00,22.13,28.62,2200.00,6,721.35,6.00,21050.00,4533.74,0.00,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
183574,True,1,34,6911.00,15.28,184.98,2100.00,2,206.00,0.00,0.00,0.00,0.00,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


In [118]:
df.to_csv('data/processed.csv', index=False)